# Rank Features (Solution)

The creator of Shapley Additive Explanations, Scott Lundberg, has written an efficient implementation that we can install and use.  We’ll be able to use this to determine both local feature importance (for a single observation) and global feature importance (for all training samples as a whole).  To aggregate local feature importance into global feature importance, we take the absolute values of the local feature importances, and then average them.

We can calculate the feature importance using sklearn and using the Shap library.

Based on the feature importances, we can think about modifying features to improve them.  Then we can re-train the model on the modified features.

Finally, we can prune the feature set to just use the most relevant features.

In [ ]:
# Note, this will install zipline and alphalens, which will take some time
import sys
!{sys.executable} -m pip install --quiet -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

In [ ]:
import os
import project_helper
from zipline.data import bundles

os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '..', '..', 'data', 'module_4_quizzes_eod')

ingest_func = bundles.csvdir.csvdir_equities(['daily'], project_helper.EOD_BUNDLE_NAME)
bundles.register(project_helper.EOD_BUNDLE_NAME, ingest_func)

print('Data Registered')

In [ ]:
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume
from zipline.utils.calendars import get_calendar


universe = AverageDollarVolume(window_length=120).top(500) 
trading_calendar = get_calendar('NYSE') 
bundle_data = bundles.load(project_helper.EOD_BUNDLE_NAME)
engine = project_helper.build_pipeline_engine(bundle_data, trading_calendar)

In [ ]:
# Test
universe_end_date = pd.Timestamp('2016-01-05', tz='UTC')

universe_tickers = engine\
    .run_pipeline(
        Pipeline(screen=universe),
        universe_end_date,
        universe_end_date)\
    .index.get_level_values(1)\
    .values.tolist()

In [ ]:
from zipline.data.data_portal import DataPortal

data_portal = DataPortal(
    bundle_data.asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=bundle_data.equity_daily_bar_reader.first_trading_day,
    equity_minute_reader=None,
    equity_daily_reader=bundle_data.equity_daily_bar_reader,
    adjustment_reader=bundle_data.adjustment_reader)

def get_pricing(data_portal, trading_calendar, assets, start_date, end_date, field='close'):
    end_dt = pd.Timestamp(end_date.strftime('%Y-%m-%d'), tz='UTC', offset='C')
    start_dt = pd.Timestamp(start_date.strftime('%Y-%m-%d'), tz='UTC', offset='C')

    end_loc = trading_calendar.closes.index.get_loc(end_dt)
    start_loc = trading_calendar.closes.index.get_loc(start_dt)

    return data_portal.get_history_window(
        assets=assets,
        end_dt=end_dt,
        bar_count=end_loc - start_loc,
        frequency='1d',
        field=field,
        data_frequency='daily')

## Make Factors


- Take the same factors we have been using:


In [ ]:
from zipline.pipeline.factors import CustomFactor, DailyReturns, Returns, SimpleMovingAverage
from zipline.pipeline.data import USEquityPricing

factor_start_date = universe_end_date - pd.DateOffset(years=3, days=2)
sector = project_helper.Sector()

def momentum_1yr(window_length, universe, sector):
    return Returns(window_length=window_length, mask=universe) \
        .demean(groupby=sector) \
        .rank() \
        .zscore()

def mean_reversion_5day_sector_neutral(window_length, universe, sector):
    return -Returns(window_length=window_length, mask=universe) \
        .demean(groupby=sector) \
        .rank() \
        .zscore()

def mean_reversion_5day_sector_neutral_smoothed(window_length, universe, sector):
    unsmoothed_factor = mean_reversion_5day_sector_neutral(window_length, universe, sector)
    return SimpleMovingAverage(inputs=[unsmoothed_factor], window_length=window_length) \
        .rank() \
        .zscore()

class CTO(Returns):
    """
    Computes the overnight return, per hypothesis from
    https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2554010
    """
    inputs = [USEquityPricing.open, USEquityPricing.close]
    
    def compute(self, today, assets, out, opens, closes):
        """
        The opens and closes matrix is 2 rows x N assets, with the most recent at the bottom.
        As such, opens[-1] is the most recent open, and closes[0] is the earlier close
        """
        out[:] = (opens[-1] - closes[0]) / closes[0]

        
class TrailingOvernightReturns(Returns):
    """
    Sum of trailing 1m O/N returns
    """
    window_safe = True
    
    def compute(self, today, asset_ids, out, cto):
        out[:] = np.nansum(cto, axis=0)

        
def overnight_sentiment(cto_window_length, trail_overnight_returns_window_length, universe):
    cto_out = CTO(mask=universe, window_length=cto_window_length)
    return TrailingOvernightReturns(inputs=[cto_out], window_length=trail_overnight_returns_window_length) \
        .rank() \
        .zscore()

def overnight_sentiment_smoothed(cto_window_length, trail_overnight_returns_window_length, universe):
    unsmoothed_factor = overnight_sentiment(cto_window_length, trail_overnight_returns_window_length, universe)
    return SimpleMovingAverage(inputs=[unsmoothed_factor], window_length=trail_overnight_returns_window_length) \
        .rank() \
        .zscore()

universe = AverageDollarVolume(window_length=120).top(500)
sector = project_helper.Sector()

pipeline = Pipeline(screen=universe)
pipeline.add(
    momentum_1yr(252, universe, sector),
    'Momentum_1YR')
pipeline.add(
    mean_reversion_5day_sector_neutral_smoothed(20, universe, sector),
    'Mean_Reversion_Sector_Neutral_Smoothed')
pipeline.add(
    overnight_sentiment_smoothed(2, 10, universe),
    'Overnight_Sentiment_Smoothed')

all_factors = engine.run_pipeline(pipeline, factor_start_date, universe_end_date)

all_factors.head()


## Add sector code

In [ ]:
pipeline.add(sector, 'sector_code')

## Universal Quant Features

* stock volatility: zipline has a custom factor called AnnualizedVolatility.  The [source code is here](https://github.com/quantopian/zipline/blob/master/zipline/pipeline/factors/basic.py) and also pasted below:


#### Annualized volatility.
Create `AnnualizedVolatility` objects for 20 day and 120 day (one month and six-month) time windows.  Remember to set the `mask` parameter to the `universe` object created earlier (this filters the stocks to match the list in the `universe`).  Convert these to ranks, and then convert the ranks to zscores.

In [ ]:
from zipline.pipeline.factors import AnnualizedVolatility
volatility_20d = AnnualizedVolatility(window_length=20, mask=universe).rank().zscore()
volatility_120d = AnnualizedVolatility(window_length=120, mask=universe).rank().zscore()
pipeline.add(volatility_20d, 'volatility_20d')
pipeline.add(volatility_120d, 'volatility_120d')

#### Average Dollar Volume feature
[AverageDollarVolume](http://www.zipline.io/appendix.html#zipline.pipeline.factors.AverageDollarVolume):
Use 20 day and 120 day `window_length`, rank and then zscore

In [ ]:
#from zipline.pipeline.factors import AverageDollarVolume # already imported earlier, but shown here for reference
adv_20d = AverageDollarVolume(window_length=20, mask=universe).rank().zscore()
adv_120d = AverageDollarVolume(window_length=120, mask=universe).rank().zscore()
pipeline.add(adv_20d, 'adv_20d')
pipeline.add(adv_120d, 'adv_120d')

### Regime Features

#### market dispersion feature

Calculate the mean returns

$\mu = \sum_{t=0}^{T}\sum_{i=1}^{N}r_{i,t}$

$\sqrt{\frac{1}{T} \sum_{t=0}^{T}  \frac{1}{N}\sum_{i=1}^{N}(r_{i,t} - \mu)^2}$

In [ ]:
class MarketDispersion(CustomFactor):
    inputs = [DailyReturns()]
    window_length = 1
    window_safe = True

    def compute(self, today, assets, out, returns):
        # returns are days in rows, assets across columns
        mean_returns = np.nanmean(returns)
        out[:] = np.sqrt(np.nanmean((returns - mean_returns)**2))
        
pipeline.add(SimpleMovingAverage(inputs=[MarketDispersion(mask=universe)], window_length=20), 'dispersion_20d')
pipeline.add(SimpleMovingAverage(inputs=[MarketDispersion(mask=universe)], window_length=120), 'dispersion_120d')

## Market volatility feature
* High and low volatility  
We'll also build a class for market volatility, which inherits from [CustomFactor](http://www.zipline.io/appendix.html?highlight=customfactor#zipline.pipeline.CustomFactor).

##### Market return
$r_{m,t} = \sum_{i=1}^{N}r_{i,t}$ for each day $t$ in `window_length`.  

##### Average market return
Also calculate the average market return over the `window_length` $W$ of days:  
$\mu_{m} = \frac{1}{N}\sum_{t=1}^{T} r_{m,t}$

#### Standard deviation of market return
Then calculate the standard deviation of the market return  
$\sigma_{m,t} = \sqrt{252 \times \frac{1}{N} \sum_{t=1}^{T}(r_{m,t} - \mu_{m})^2 } $ 


In [ ]:
class MarketVolatility(CustomFactor):
    inputs = [DailyReturns()]
    window_length = 1  # We'll want to set this in the constructor when creating the object.
    window_safe = True
    
    
    def compute(self, today, assets, out, returns):
        DAILY_TO_ANNUAL_SCALAR = 252.  # 252 trading days in a year
        """
        For each row (each row represents one day of returns), 
        calculate the average of the cross-section of stock returns
        So that market_returns has one value for each day in the window_length
        So choose the appropriate axis (please see hints above)
        """
        mkt_returns = np.nanmean(returns, axis=1) 
        
        """ 
        Calculate the mean of market returns
        """
        mkt_returns_mu = np.nanmean(mkt_returns)
        
        """
        Calculate the standard deviation of the market returns, then annualize them.
        """
        out[:] = np.sqrt(DAILY_TO_ANNUAL_SCALAR * np.nanmean((mkt_returns-mkt_returns_mu)**2))
        
# create market volatility features using one month and six-month windows
market_vol_20d = MarketVolatility(window_length=20)
market_vol_120d = MarketVolatility(window_length=120)

# add market volatility features to pipeline
pipeline.add(market_vol_20d, 'market_vol_20d')
pipeline.add(market_vol_120d, 'market_vol_120d')

#### Run pipeline to calculate features


In [ ]:
all_factors = engine.run_pipeline(pipeline, factor_start_date, universe_end_date)
all_factors.head(2)

### Make Date Parts
* we make colums to for the trees to split on that might capture trader/investor behavior due to calendar anomalies.
* We can get the dates from the index of the dataframe that is returned from running the pipeline

## January, December
* Create a numpy array that has 1 when the month is January, and 0 otherwise.  Store it as a column in the all_factors dataframe.
* Add another similar column to indicate when the month is December

In [ ]:
all_factors['is_January'] = (all_factors.index.get_level_values(0).month == 1).astype(int)
all_factors['is_December'] = (all_factors.index.get_level_values(0).month == 12).astype(int)

## Weekday, quarter
* add columns to the all_factors dataframe that specify the weekday, quarter and year

In [ ]:
all_factors['weekday'] = all_factors.index.get_level_values(0).weekday
all_factors['quarter'] = all_factors.index.get_level_values(0).quarter
all_factors['year'] = all_factors.index.get_level_values(0).year

## Start and end-of features

* The start and end of the week, month, and quarter may have structural differences in trading activity.
* [Pandas.date_range](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.date_range.html) takes the start_date, end_date, and frequency.
* The [frequency](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases) for end of month is `BM`.

In [ ]:
all_factors['month_end'] = all_factors.index.get_level_values(0).isin(pd.date_range(start=factor_start_date, end=universe_end_date, freq='BM')).astype(int)
all_factors['month_start'] = all_factors.index.get_level_values(0).isin(pd.date_range(start=factor_start_date, end=universe_end_date, freq='BMS')).astype(int)
all_factors['qtr_end'] = all_factors.index.get_level_values(0).isin(pd.date_range(start=factor_start_date, end=universe_end_date, freq='BQ')).astype(int)
all_factors['qtr_start'] = all_factors.index.get_level_values(0).isin(pd.date_range(start=factor_start_date, end=universe_end_date, freq='BQS')).astype(int)

In [ ]:
all_factors.columns

In [ ]:
features = list(all_factors.columns)
features

### Make Target




In [ ]:
pipeline_target = Pipeline(screen=universe)

#### Example

We'll convert returns into 5-quantiles.

In [ ]:
return_5d_5q = Returns(window_length=5, mask=universe).quantiles(5)
return_5d_5q

In [ ]:
pipeline_target.add(return_5d_5q, 'return_5d_5q')

In [ ]:
targets_df = engine.run_pipeline(pipeline_target, factor_start_date, universe_end_date)
targets_df.head()

In [ ]:
targets_df.columns

In [ ]:
target_label = 'return_5d_5q'

In [ ]:
all_factors.index.get_level_values(1)

In [ ]:
targets_df.index.get_level_values(1)

### Split into training, validation and test

In [ ]:
def split_into_sets(data, set_sizes):
    assert np.sum(set_sizes) == 1
    
    last_i = 0
    sets = []
    for set_size in set_sizes:
        set_n = int(len(data) * set_size)
        sets.append(data[last_i:last_i + set_n])
        last_i = last_i + set_n
        
    return sets

def split_by_index(df, index_level, sets):
    set_indicies = split_into_sets(df.index.levels[index_level], sets)
    
    return [df.loc[indicies[0]:indicies[-1]] for indicies in set_indicies]

In [ ]:
# put the features and target into one dataframe before 
# running dropna, so that the rows match.
tmp = all_factors.copy()
tmp [target_label] = targets_df[target_label]
tmp = tmp.dropna()
X = tmp[features]
y = tmp[target_label]

X_train, X_valid, X_test = split_by_index(X, 0, [0.6, 0.2, 0.2])
y_train, y_valid, y_test = split_by_index(y, 0, [0.6, 0.2, 0.2])

In [ ]:
X_train.shape

In [ ]:
y_train.shape

## Fit a random forest

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, ExtraTreesRegressor, RandomForestRegressor
from sklearn.metrics import log_loss

In [ ]:
clf = RandomForestClassifier(
        n_estimators=10,
        max_features='sqrt',
        min_samples_split=5000,
        bootstrap=True,
        oob_score=True,
        n_jobs=-1,
        criterion='entropy',
        verbose=0,
        random_state=0
    )
clf.fit(X_train, y_train)

# Rank features by Feature importance (sklearn)

We'll define a function that uses the built in sklearn feature importances, and sorts the features by their feature importance.

Note that [numpy.argsort](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html) returns a list of the original index locations of a list, in the order that would make them sorted in ascending order.

In [ ]:
np.argsort([10,30,20])

One way to reverse the order of a list or array is to use the notation `[::-1]`

In [ ]:
tmp = [3,2,1]
tmp[::-1]

In [ ]:
def model_importances(m, features):
    # TODO: get the feature importances from the model
    importances = m.feature_importances_
    
    # TODO: sort the importances in descending order, and store the indices of that sort
    indices = np.argsort(importances)[::-1]
    """
    Iterate through the features, starting with the ones with the highest feature importances
    """
    features_ranked = []
    for f in range(X_train.shape[1]):
        print("%d. %s (%d) (%f)" % (f+1,features[indices[f]], indices[f], importances[indices[f]]))
        features_ranked.append(features[indices[f]])

    return features_ranked

#### See ranking of features according to sklearn's feature_importances

In [ ]:
features_skl = model_importances(clf, features)

## Using Shap library

We'll also use the Shap library to determine feature importance.

In [ ]:
import shap
shap.initjs() #initialize javascript to enable visualizations

#### Shap outputs 
https://shap.readthedocs.io/en/latest/

```
shap_values(X, y=None, tree_limit=-1, approximate=False)

X:
A matrix of samples (# samples x # features) on which to explain the model’s output.

tree_limit:
Limit the number of trees used by the model.

approximate:
Run fast, but only roughly approximate the Tree SHAP values
```

>**For models with a single output this returns a matrix of SHAP values (# samples x # features).** 

>Each row sums to the difference between the model output for that sample and the expected value of the model output (which is stored in the expected_value attribute of the explainer when it is constant). 

>**For models with vector outputs this returns a list of such matrices, one for each output.**




In [ ]:
# this will take a few seconds to run
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_train, tree_limit=5)

The classifier has 5 distinct outputs (5 quantiles).  
So the shap_values is a list of 5 numpy arrays.

In [ ]:
set(y_train)

In [ ]:
len(shap_values)

Each element of shap_values has one row for each training data point, and one column for each feature.

In [ ]:
# features, data points
shap_values[0].shape

## local to global feature importance

SHAP calculates local feature importance for every training observation (every row).  
To calculate global feature importance, take the absolute values of the local feature importances and then take the average across all samples.

$GlobalImportance_{i} = \frac{1}{N}\sum_{j=1}^{N} |LocalImportance_{i,j}|$  

Where there are N samples, and $i$ denotes a particular feature.

We can use the built in function to plot the features sorted by global feature importance.  This is taking the average of the absolute values of the shapley values for each feature, to get the global feature importance.

In [ ]:
# mean of absolute values for each feature to get global feature importance
shap.summary_plot(shap_values, X_train, plot_type="bar")

Note that the plot shows the first 20 features.  We can write our own function to calculate global feature importance, so that we can see the global feature importance of all features.

## Rank features using SHAP

There are a couple classes, one for each quantile.  So the list returned by `shap.shap.TreeExplainer.shap_values()` has one element for each of those classes.  We'll explore how to get the absolute values and then average of those absolute values, for each of the features.  Then we can put this into a function.

Remember, here's the formula to aggregate local feature importances into global feature importance:

$GlobalImportance_{i} = \frac{1}{N}\sum_{j=1}^{N} |LocalImportance_{i,j}|$  

We can concatenate the 2D arrays in the list `shap_values`.

In [ ]:
tmp1 = np.concatenate(shap_values)
tmp1.shape

Take the absolute values

In [ ]:
tmp2 = np.abs(tmp1)
tmp2

Take the average for each column

In [ ]:
tmp3 = np.nanmean(tmp2,axis=0)
tmp3

## Quiz

Implement the function that calculates global feature importance using shapley values, and sorts the features by importance.

In [ ]:
"""Challenge: try implementing the function yourself! """

def model_shap_importances(model, features,X):
    pass

You can also use the starter code below, if you prefer:

In [ ]:
def model_shap_importances(model, features,X):
    """
    Note that the observations should be numeric (integer or float).
    So booleans should be converted to 1 (True) and 0 (False) 
    """
    # TODO: calculate shap values
    shap_values = shap.TreeExplainer(model).shap_values(X, tree_limit=5)
    
    # TODO: concatenate the shap values into one matrix
    shap_values_matrix = np.concatenate(shap_values)
    
    # TODO: take the absolute values
    shap_abs = np.abs(shap_values_matrix)
    
    # TODO: Take the average for each feature (each column)
    global_importances = np.nanmean(shap_abs, axis=0)
        
    # TODO: get the indices sorted in descending order of global feature importance
    indices = np.argsort(global_importances)[::-1]
    features_ranked = []
    for f in range(X.shape[1]):
        print("%d. %s (%d) (%f)" % (f+1,features[indices[f]], indices[f], global_importances[indices[f]]))
        features_ranked.append(features[indices[f]])
        
    return features_ranked

In [ ]:
# this will take a few seconds to run
features_ranked = model_shap_importances(clf,features,X_train)

In [ ]:
features_ranked

## Discussion on sector

- Random forests can still work with categorical features that are numbers.  For instance, to filter features by sector '5', it's possible for a tree to split on sector < 6, and then sector > 4.  One of the reasons tree-based models are great is because they can still try to interpret data that hasn't been fully cleaned or processed.  However it's still a best practice to one-hot encode categorical features, as this will help to reduce noise, and hopefully help the model's performance.

## Sector category names

You'll one-hot encode the sector with sector labels in the project.  Please see some code that can use when you assign category labels to each sector.

In [ ]:
## Sector Labels
sector_names = pd.read_csv('sector_names.csv')
sector_names = sector_names[['Sector','sector_id']]
sector_names = sector_names.drop_duplicates()
sector_names = sector_names.append(pd.DataFrame([['no sector assigned',-1]], columns = sector_names.columns))
sector_names

In [ ]:
# use this dataframe to get the sector name by sector id
# here's an example
tmp = sector_names.loc[sector_names['sector_id'] == 9]['Sector'].values[0]
tmp

## One-hot encode other features?
Are there other features here that you could also one-hot encode?  You can one-hot encode these other features in the project!

## date features

The low frequency date parts (end of month, end of quarter etc.) have low importance. This should not be surprising since we are looking at only a couple years of training history and there are only, say, 4 quarters in a year. 

End of month trading activity may occur some days before the last business day of the month.  To better capture what we think of as end-of the month trading, we can try including the last 5 business days of the month.  Similarly, we can try including the last two weeks of each quarter.

## Date features helper code
You may find some of these functions useful in the project!

* We can use [BDay](https://pandas.pydata.org/pandas-docs/stable/timeseries.html) to offset our date_range by a specified number of business days
* Also, check out a list of [frequencies](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases) to choose from, such as `M`, `Q`.

#### Example

In [ ]:
from pandas.tseries.offsets import BDay

In [ ]:
tmp = all_factors.index.get_level_values(0)
tmp

In [ ]:
tmp_1 = tmp + BDay(-1)
tmp_1

Notice how adding `Bday(-1)` to the DateTimeIndex `tmp` made another DateTimeIndex with the second to last business day of each month.

#### Union
DatetimeIndex has a [union](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DatetimeIndex.union.html) function that merges one DatetimeIndex with another

In [ ]:
tmp_1.union(tmp)

You may find this code useful in the project!

## Prune features

In the project, you'll try to improve some of your features, and then check how their feature importance changes (if at all).  Then you can prune your feature list and choose the ones that you want to use in your model.

## Solution

[solution notebook](feature_importance_solution.ipynb)